In [9]:
# increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd

### Getting the demo data

In [151]:
demo_df = pd.read_csv('../data/QA/congresspeople_demo.txt', sep='|')

In [152]:
demo_df.shape

(1800, 14)

In [49]:
demo_df['Full Name'].nunique()

1790

### Getting the speaking data

In [67]:
# reading file with all speakers in one place
filepath = "../EDA/full_speakermap.txt"

In [68]:
with open(filepath) as f:
    speakermap = []
    for line in f:
        line = line.strip().split("\t")
        speakermap.append(line)

In [69]:
colnames = ['speakerid', 'Congress', 'Full Name', 'Chamber', 'State', 'Gender', 'Party', 'SpeachCount']
speakermap = pd.DataFrame(speakermap, columns=colnames)
speakermap.shape

(9845, 8)

In [70]:
speakermap.head()

,speakerid,Congress,Full Name,Chamber,State,Gender,Party,SpeachCount
0,97105301,97,"PERCY, CHARLES",S,IL,M,R,2050
1,97105071,97,"HUDDLESTON, WALTER",S,KY,M,D,563
2,97106761,97,"JACKSON, HENRY",S,WA,M,D,560
3,97106981,97,"STAFFORD, ROBERT",S,VT,M,R,497
4,97104811,97,"BAKER, HOWARD",S,TN,M,R,15456


In [71]:
speakermap['Congress'] = pd.to_numeric(speakermap['Congress'])

### Fixing names for father/son

In [72]:
speakermap['Full Name'].nunique()

1781

In [73]:
speakermap.iloc[[(speakermap['Full Name']=='FORD, HAROLD') & (speakermap['Congress']>=105)],[speakermap.columns.get_loc('Full Name')]]='FORD, HAROLD E., JR.'
speakermap.iloc[[(speakermap['Full Name']=='KENNEDY, JOSEPH') & (speakermap['Congress']>=113)],[speakermap.columns.get_loc('Full Name')]]='KENNEDY, JOSEPH P., III'
speakermap.iloc[[(speakermap['Full Name']=='JONES, WALTER') & (speakermap['Congress']>=104)],[speakermap.columns.get_loc('Full Name')]]='JONES, WALTER B., JR.'
speakermap.iloc[[(speakermap['Full Name']=='HUNTER, DUNCAN') & (speakermap['Congress']>=111)],[speakermap.columns.get_loc('Full Name')]]='HUNTER, DUNCAN D.'
speakermap.iloc[[(speakermap['Full Name']=='PAYNE, DONALD') & (speakermap['Congress']>=112)],[speakermap.columns.get_loc('Full Name')]]='PAYNE, DONALD M., JR.'
speakermap.iloc[[(speakermap['Full Name']=='DUNCAN, JOHN') & (speakermap['Congress']>=100)],[speakermap.columns.get_loc('Full Name')]]='DUNCAN, JOHN J., JR.'
speakermap.iloc[[(speakermap['Full Name']=='MACK, CONNIE') & (speakermap['Congress']<109)],[speakermap.columns.get_loc('Full Name')]]='MACK, CONNIE, III'
speakermap.iloc[[(speakermap['Full Name']=='RHODES, JOHN') & (speakermap['Congress']>=100)],[speakermap.columns.get_loc('Full Name')]]='RHODES, JOHN J., III'
speakermap.iloc[[(speakermap['Full Name']=='PERKINS, CARL') & (speakermap['Congress']>=99)],[speakermap.columns.get_loc('Full Name')]]='PERKINS, CARL C.'

In [74]:
speakermap['Full Name'].nunique()

1790

In [75]:
errors = ['FORD, HAROLD','KENNEDY, JOSEPH','JONES, WALTER','HUNTER, DUNCAN',
         'PAYNE, DONALD','DUNCAN, JOHN','MACK, CONNIE','RHODES, JOHN','PERKINS, CARL']
fixes = ['FORD, HAROLD E., JR.','KENNEDY, JOSEPH P., III','JONES, WALTER B., JR.','HUNTER, DUNCAN D.',
         'PAYNE, DONALD M., JR.','DUNCAN, JOHN J., JR.','MACK, CONNIE, III','RHODES, JOHN J., III','PERKINS, CARL C.']

In [76]:
for e in errors:
    print(e, speakermap[speakermap['Full Name']==e]['Congress'].values)

FORD, HAROLD [ 97  98  99 100 101 102 103 104]
KENNEDY, JOSEPH [100 101 102 103 104 105]
JONES, WALTER [ 97  98  99 100 101 102]
HUNTER, DUNCAN [ 97  98  99 100 101 102 103 104 105 106 107 108 109 110]
PAYNE, DONALD [101 102 103 104 105 106 107 108 109 110 111]
DUNCAN, JOHN [97 98 99]
MACK, CONNIE [109 110 111 112]
RHODES, JOHN [97]
PERKINS, CARL [97]


In [77]:
for f in fixes:
    print(f, speakermap[speakermap['Full Name']==f]['Congress'].values)

FORD, HAROLD E., JR. [105 106 107 108 109]
KENNEDY, JOSEPH P., III [113 114]
JONES, WALTER B., JR. [104 105 106 107 108 109 110 111 112 113 114]
HUNTER, DUNCAN D. [111 112 113 114]
PAYNE, DONALD M., JR. [113 114]
DUNCAN, JOHN J., JR. [100 101 102 103 104 105 106 107 108 109 110 111 112 113 114]
MACK, CONNIE, III [ 98  99 100 101 102 103 104 105 106]
RHODES, JOHN J., III [100 101 102]
PERKINS, CARL C. [ 99 100 101 102]


In [148]:
pwd

'/tf/notebooks/EDA'

### Merging demo and speakermaps

In [78]:
keepcols = ['Full Name', 'State', 'List of Congresses', 'Ethnicity', 'BirthYear']
full_df = speakermap.merge(demo_df[keepcols], how='left', on=['Full Name', 'State'])
full_df.shape

(9845, 11)

In [79]:
full_df.head()

,speakerid,Congress,Full Name,Chamber,State,Gender,Party,SpeachCount,List of Congresses,Ethnicity,BirthYear
0,97105301,97,"PERCY, CHARLES",S,IL,M,R,2050,97 98,W,1919
1,97105071,97,"HUDDLESTON, WALTER",S,KY,M,D,563,97 98,W,1926
2,97106761,97,"JACKSON, HENRY",S,WA,M,D,560,97 98,W,1912
3,97106981,97,"STAFFORD, ROBERT",S,VT,M,R,497,97 98 99 100,W,1913
4,97104811,97,"BAKER, HOWARD",S,TN,M,R,15456,97 98,W,1925


In [80]:
full_df['Full Name'].nunique()

1790

In [81]:
full_df.Congress = pd.to_numeric(full_df.Congress)
full_df.SpeachCount = pd.to_numeric(full_df.SpeachCount)
full_df.BirthYear = pd.to_numeric(full_df.BirthYear)

In [82]:
# creating congress year variable
full_df['CongressYear'] = 1982 + (full_df.Congress - 97)*2

In [83]:
full_df.head()

,speakerid,Congress,Full Name,Chamber,State,Gender,Party,SpeachCount,List of Congresses,Ethnicity,BirthYear,CongressYear
0,97105301,97,"PERCY, CHARLES",S,IL,M,R,2050,97 98,W,1919,1982
1,97105071,97,"HUDDLESTON, WALTER",S,KY,M,D,563,97 98,W,1926,1982
2,97106761,97,"JACKSON, HENRY",S,WA,M,D,560,97 98,W,1912,1982
3,97106981,97,"STAFFORD, ROBERT",S,VT,M,R,497,97 98 99 100,W,1913,1982
4,97104811,97,"BAKER, HOWARD",S,TN,M,R,15456,97 98,W,1925,1982


In [84]:
full_df['CongressYear'].value_counts().sort_index()

1982    545
1984    543
1986    543
1988    545
1990    550
1992    550
1994    550
1996    546
1998    548
2000    541
2002    544
2004    542
2006    545
2008    555
2010    556
2012    547
2014    553
2016    542
Name: CongressYear, dtype: int64

In [85]:
full_df['Age'] = full_df['CongressYear'] - full_df['BirthYear']
full_df['Age'].describe()

count    9845.000000
mean       55.809446
std        10.603989
min        28.000000
25%        48.000000
50%        56.000000
75%        63.000000
max       100.000000
Name: Age, dtype: float64

In [31]:
# the 100-year old congress person was for real
full_df[full_df['Age']==100]

,speakerid,Congress,Full Name,Chamber,State,Gender,Party,SpeachCount,List of Congresses,Ethnicity,BirthYear,CongressYear,Age
5471,107112201,107,"THURMOND, J.",S,SC,M,R,108,97 98 99 100 101 102 103 104 105 106 107,W,1902,2002,100


In [153]:
full_df.to_csv('../data/QA/full_speakermap_demo.txt', sep='|', index=False)

## EDA

### Target Variables

In [98]:
full_df['Age'].median()

56.0

In [138]:
# get median by congress
med = full_df[['Age','Congress']].groupby(['Congress']).median().reset_index()
med.columns = ['Congress', 'Age_med']
full_df = full_df.merge(med, how='left', on='Congress')
full_df['Age_med'].describe()

count    9845.000000
mean       55.508075
std         3.145272
min        51.000000
25%        53.000000
50%        55.000000
75%        59.000000
max        60.000000
Name: Age_med, dtype: float64

In [139]:
full_df['Age_lt_med'] = full_df.apply(lambda row: 1 if row['Age'] < row['Age_med'] else 0, axis=1)
full_df['Age_lt_med'].mean()

0.4824784154393093

In [140]:
full_df['Age_lt_med'].sum()

4750

In [141]:
list(full_df['Ethnicity'].value_counts().index)

['W', 'B', 'H', 'AAPIA', 'ME', 'B/AAPIA', 'B/H', 'H/ME']

In [114]:
full_df['NonWhite'] = full_df.apply(lambda row: 0 if row['Ethnicity'] == 'W' else 1, axis=1)
full_df['NonWhite'].mean()

0.12686642965972575

In [115]:
full_df['NonWhite'].sum()

1249

In [116]:
full_df['Female'] = full_df.apply(lambda row: 1 if row['Gender'] == 'F' else 0, axis=1)
full_df['Female'].mean()

0.11467750126968004

In [117]:
full_df['Female'].sum()

1129

In [119]:
pd.crosstab(full_df['NonWhite'], full_df['Female'])

Female,0,1
NonWhite,,
0,7779,817
1,937,312


In [120]:
list(full_df['Party'].value_counts().index)

['D', 'R', 'I', 'N', 'A', 'P']

In [121]:
pd.crosstab(full_df['NonWhite'], full_df['Party'])

Party,A,D,I,N,P,R
NonWhite,,,,,,
0,0,4065,26,0,0,4505
1,1,1093,2,2,1,150


In [122]:
pd.crosstab(full_df['Female'], full_df['Party'])

Party,A,D,I,N,P,R
Female,,,,,,
0,1,4375,28,2,1,4309
1,0,783,0,0,0,346


In [123]:
pd.crosstab(full_df['Age_lt_med'], full_df['Party'])

Party,A,D,I,N,P,R
Age_lt_med,,,,,,
0,0,2686,21,0,0,2249
1,1,2472,7,2,1,2406


### Speach Counts

In [97]:
# total number of speaches
full_df['SpeachCount'].sum()

2585807

In [87]:
# average speach count per person by congress
full_df[['SpeachCount','Congress']].groupby(['Congress']).mean()

,SpeachCount
Congress,
97,370.741284
98,370.532228
99,371.801105
100,352.379817
101,303.445455
102,295.401818
103,290.798182
104,354.135531
105,257.728102


### Congress People Age

In [155]:
full_df[['Age','Party']].groupby(['Party']).mean()

,Age
Party,
A,40.000000
D,56.426328
I,63.392857
N,48.000000
P,42.000000
R,55.090011


In [154]:
mean_age = full_df[['Age','Congress']].groupby(['Congress']).mean()
med_age = full_df[['Age','Congress']].groupby(['Congress']).median()
min_age = full_df[['Age','Congress']].groupby(['Congress']).min()
max_age = full_df[['Age','Congress']].groupby(['Congress']).max()
temp = pd.concat([min_age, mean_age, med_age, max_age], axis=1)
temp.columns = ['Min', 'Mean', 'Med', 'Max']
temp

,Min,Mean,Med,Max
Congress,,,,
97,29,50.992661,51.0,82
98,30,51.569061,51.0,84
99,29,52.482505,52.0,86
100,31,53.332110,52.0,88
101,32,54.172727,53.0,90
102,32,55.269091,54.0,90
103,32,54.474545,53.5,92
104,29,54.195971,54.0,94
105,28,54.624088,54.0,96


In [143]:
full_df[['NonWhite', 'SpeachCount']].groupby(['NonWhite']).sum()

,SpeachCount
NonWhite,
0,2377810
1,207997


In [145]:
full_df[['Female', 'SpeachCount']].groupby(['Female']).sum()

,SpeachCount
Female,
0,2385762
1,200045


In [146]:
full_df[['Age_lt_med', 'SpeachCount']].groupby(['Age_lt_med']).sum()

,SpeachCount
Age_lt_med,
0,1719089
1,866718


### Party-leval Analysis

In [88]:
temp = full_df[['speakerid','Congress','Party']].groupby(['Congress','Party']).count().reset_index()
pd.pivot_table(temp, values='speakerid', index='Congress', columns='Party', fill_value=0)

Party,A,D,I,N,P,R
Congress,,,,,,
97,0,293,2,1,0,249
98,0,321,0,1,0,221
99,0,308,0,0,0,235
100,0,321,0,0,0,224
101,0,325,0,0,0,225
102,0,334,1,0,0,215
103,0,321,1,0,0,228
104,0,253,2,0,0,291
105,0,262,1,0,0,285


#### Speach Count per Person by Party

In [89]:
temp1 = full_df[['SpeachCount','Congress','Party']].groupby(['Congress','Party']).sum().reset_index()
temp2 = full_df[['speakerid','Congress','Party']].groupby(['Congress','Party']).count().reset_index()
temp = temp1.merge(temp2, how='left', on=['Congress','Party'])
temp['SpeachesPerPerson'] = temp.SpeachCount / temp.speakerid
party = pd.pivot_table(temp, values='SpeachesPerPerson', index='Congress', columns='Party', fill_value=0)
party['D_minus_R'] = party.D - party.R
party

Party,A,D,I,N,P,R,D_minus_R
Congress,,,,,,,
97,0,324.597270,447.00,186,0,425.168675,-100.571405
98,0,296.146417,0.00,148,0,479.583710,-183.437293
99,0,313.840909,0.00,0,0,447.765957,-133.925048
100,0,335.199377,0.00,0,0,377.000000,-41.800623
101,0,295.575385,0.00,0,0,314.813333,-19.237949
102,0,284.023952,127.00,0,0,313.860465,-29.836513
103,0,275.900312,141.00,0,0,312.429825,-36.529513
104,0,351.715415,275.00,0,0,356.783505,-5.068090
105,0,231.274809,307.00,0,0,281.873684,-50.598875


### Ethnicity Count by Congress

In [90]:
full_df.Ethnicity.value_counts()

W          8596
B           613
H           376
AAPIA       158
ME           41
B/AAPIA      19
B/H          18
H/ME          6
Name: Ethnicity, dtype: int64

In [91]:
temp = full_df[['speakerid','Congress','Ethnicity']].groupby(['Congress','Ethnicity']).count().reset_index()
pd.pivot_table(temp, values='speakerid', index='Congress', columns='Ethnicity', fill_value=0)

Ethnicity,AAPIA,B,B/AAPIA,B/H,H,H/ME,ME,W
Congress,,,,,,,,
97,8,17,1,1,8,0,3,507
98,7,20,1,1,12,0,3,499
99,7,19,1,1,13,0,2,500
100,8,21,1,1,13,0,1,499
101,10,23,1,1,14,0,1,499
102,7,25,1,1,15,0,1,499
103,8,39,1,1,20,0,2,478
104,9,41,1,1,19,0,2,472
105,8,40,1,1,21,1,2,473


In [92]:
temp1 = full_df[['SpeachCount','Congress','Ethnicity']].groupby(['Congress','Ethnicity']).sum().reset_index()
temp1 = pd.pivot_table(temp1, values='SpeachCount', index='Congress', columns='Ethnicity', fill_value=0)
temp1['NW'] = temp1['AAPIA']+temp1['B']+temp1['B/AAPIA']+temp1['B/H']+temp1['H']+temp1['H/ME']+temp1['ME']
temp2 = full_df[['speakerid','Congress','Ethnicity']].groupby(['Congress','Ethnicity']).count().reset_index()
temp2 = pd.pivot_table(temp2, values='speakerid', index='Congress', columns='Ethnicity', fill_value=0)
temp2['NW'] = temp2['AAPIA']+temp2['B']+temp2['B/AAPIA']+temp2['B/H']+temp2['H']+temp2['H/ME']+temp2['ME']
temp = pd.concat([temp1[['W','NW']], temp2[['W','NW']]], axis=1)
temp.columns = ['W_cnt','NW_cnt','W_spk','NW_spk']
temp['W_cnt_p_spk'] = temp['W_cnt']/temp['W_spk']
temp['NW_cnt_p_spk'] = temp['NW_cnt']/temp['NW_spk']
temp['NW_minus_W'] = temp['NW_cnt_p_spk'] - temp['W_cnt_p_spk']
temp[['W_spk','W_cnt_p_spk','NW_spk','NW_cnt_p_spk','NW_minus_W']]

,W_spk,W_cnt_p_spk,NW_spk,NW_cnt_p_spk,NW_minus_W
Congress,,,,,
97,507,382.749507,38,210.526316,-172.223191
98,499,384.561122,44,211.431818,-173.129304
99,500,384.730000,43,221.465116,-163.264884
100,499,367.703407,45,188.911111,-178.792296
101,499,303.645291,50,306.220000,2.574709
102,499,299.821643,50,256.580000,-43.241643
103,478,305.817992,71,192.028169,-113.789823
104,472,380.025424,73,189.191781,-190.833643
105,473,273.073996,74,154.540541,-118.533455


### Gender Counts by Congress

In [93]:
full_df.Gender.value_counts()

M    8716
F    1129
Name: Gender, dtype: int64

In [94]:
temp = full_df[['speakerid','Congress','Gender']].groupby(['Congress','Gender']).count().reset_index()
pd.pivot_table(temp, values='speakerid', index='Congress', columns='Gender', fill_value=0)

Gender,F,M
Congress,,
97,22,523
98,24,519
99,25,518
100,26,519
101,31,519
102,32,518
103,53,497
104,58,488
105,65,483


In [95]:
temp1 = full_df[['SpeachCount','Congress','Gender']].groupby(['Congress','Gender']).sum().reset_index()
temp1 = pd.pivot_table(temp1, values='SpeachCount', index='Congress', columns='Gender', fill_value=0)
temp2 = full_df[['speakerid','Congress','Gender']].groupby(['Congress','Gender']).count().reset_index()
temp2 = pd.pivot_table(temp2, values='speakerid', index='Congress', columns='Gender', fill_value=0)
temp = pd.concat([temp1, temp2], axis=1)
temp.columns = ['F_cnt','M_cnt','F_spk','M_spk']
temp['F_cnt_p_spk'] = temp['F_cnt']/temp['F_spk']
temp['M_cnt_p_spk'] = temp['M_cnt']/temp['M_spk']
temp['F_minus_M'] = temp['F_cnt_p_spk'] - temp['M_cnt_p_spk']
temp[['F_spk','F_cnt_p_spk','M_spk','M_cnt_p_spk','F_minus_M']]

,F_spk,F_cnt_p_spk,M_spk,M_cnt_p_spk,F_minus_M
Congress,,,,,
97,22,172.409091,523,379.084130,-206.675039
98,24,167.750000,519,379.909441,-212.159441
99,25,190.160000,518,380.567568,-190.407568
100,26,176.769231,519,361.177264,-184.408033
101,31,161.516129,519,311.922929,-150.406800
102,32,169.093750,518,303.204633,-134.110883
103,53,182.566038,497,302.340040,-119.774003
104,58,280.224138,488,362.920082,-82.695944
105,65,190.615385,483,266.759834,-76.144450
